# Lab 5: Surface Fluxes and Atmospheric Interactions

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MichalBrezny/LSPD-lab/blob/main/labs/Lab05_Surface_Fluxes_Atmospheric_Interactions.ipynb)

## Objectives
- Integrate energy balance components
- Understand turbulent flux measurements and calculations
- Analyze boundary layer development
- Explore land-atmosphere feedbacks

## Background
The land surface and atmosphere are tightly coupled through exchanges of energy, water, and momentum. Understanding these interactions is crucial for weather prediction, climate modeling, and ecosystem management.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import fsolve

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Physical constants
VON_KARMAN = 0.41
GRAVITY = 9.81  # m/s²
SPECIFIC_HEAT = 1005  # J/(kg·K)
AIR_DENSITY = 1.2  # kg/m³

print("Libraries imported successfully!")

## Part 1: Complete Energy Balance

The complete surface energy balance:

$R_n = H + LE + G$

We'll simulate all components and verify closure.

In [ ]:
def complete_energy_balance(hour, surface_type='grass'):
    """
    Calculate complete energy balance for given hour and surface type.
    
    Parameters:
    - hour: hour of day (0-24)
    - surface_type: type of surface
    
    Returns:
    - Dictionary with all energy balance components
    """
    # Net radiation (simplified)
    Rn = 400 * np.sin(np.pi * (hour - 6) / 12)
    if Rn < 0:
        Rn = -50
    
    # Surface-specific partitioning
    partitioning = {
        'grass': {'H_frac': 0.25, 'LE_frac': 0.60, 'G_frac': 0.15},
        'forest': {'H_frac': 0.20, 'LE_frac': 0.70, 'G_frac': 0.10},
        'desert': {'H_frac': 0.60, 'LE_frac': 0.10, 'G_frac': 0.30},
        'water': {'H_frac': 0.10, 'LE_frac': 0.85, 'G_frac': 0.05}
    }
    
    if Rn > 0:
        p = partitioning[surface_type]
        H = p['H_frac'] * Rn
        LE = p['LE_frac'] * Rn
        G = p['G_frac'] * Rn
    else:
        # Nighttime
        H = 0.6 * Rn
        LE = 0
        G = 0.4 * Rn
    
    return {
        'Rn': Rn,
        'H': H,
        'LE': LE,
        'G': G,
        'Closure': Rn - (H + LE + G)
    }

# Calculate for multiple surfaces
hours = np.arange(0, 24, 0.5)
surfaces = ['grass', 'forest', 'desert', 'water']

energy_balance_data = {}
for surface in surfaces:
    data = {'Rn': [], 'H': [], 'LE': [], 'G': [], 'Closure': []}
    
    for hour in hours:
        result = complete_energy_balance(hour, surface)
        for key in data.keys():
            data[key].append(result[key])
    
    for key in data.keys():
        data[key] = np.array(data[key])
    
    energy_balance_data[surface] = data

print("Energy balance calculated for all surfaces!")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot for each surface type
for idx, surface in enumerate(surfaces):
    ax = axes[idx // 2, idx % 2]
    data = energy_balance_data[surface]
    
    ax.plot(hours, data['Rn'], 'k-', linewidth=2.5, label='Rn', zorder=5)
    ax.plot(hours, data['H'], 'r--', linewidth=2, label='H')
    ax.plot(hours, data['LE'], 'b--', linewidth=2, label='LE')
    ax.plot(hours, data['G'], 'g--', linewidth=2, label='G')
    
    ax.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
    ax.set_xlabel('Hour of Day', fontsize=11)
    ax.set_ylabel('Energy Flux (W/m²)', fontsize=11)
    ax.set_title(f'Energy Balance - {surface.capitalize()}', fontsize=12, fontweight='bold')
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 2: Turbulent Fluxes and Stability

Sensible heat flux can be calculated using flux-gradient relationships:

$H = \rho c_p \frac{u_* \theta_*}{\phi_h}$

where:
- $u_*$ = friction velocity
- $\theta_*$ = temperature scale
- $\phi_h$ = stability correction function

In [ ]:
def monin_obukhov_length(u_star, T, H):
    """
    Calculate Monin-Obukhov length.
    
    Parameters:
    - u_star: friction velocity (m/s)
    - T: temperature (K)
    - H: sensible heat flux (W/m²)
    
    Returns:
    - L: Monin-Obukhov length (m)
    """
    if abs(H) < 1:
        return 1e6  # Neutral conditions
    
    theta_star = -H / (AIR_DENSITY * SPECIFIC_HEAT * u_star)
    L = u_star**2 * T / (VON_KARMAN * GRAVITY * theta_star)
    
    return L

def stability_parameter(z, L):
    """
    Calculate stability parameter.
    
    Parameters:
    - z: height (m)
    - L: Monin-Obukhov length (m)
    
    Returns:
    - zeta: stability parameter
    """
    if abs(L) > 1e5:
        return 0  # Neutral
    return z / L

def stability_correction(zeta):
    """
    Calculate stability correction functions.
    
    Parameters:
    - zeta: stability parameter
    
    Returns:
    - psi_m: correction for momentum
    - psi_h: correction for heat
    """
    if zeta < 0:  # Unstable
        x = (1 - 16 * zeta)**0.25
        psi_m = 2 * np.log((1 + x) / 2) + np.log((1 + x**2) / 2) - 2 * np.arctan(x) + np.pi / 2
        psi_h = 2 * np.log((1 + x**2) / 2)
    elif zeta > 0:  # Stable
        psi_m = -5 * zeta
        psi_h = -5 * zeta
    else:  # Neutral
        psi_m = 0
        psi_h = 0
    
    return psi_m, psi_h

# Calculate stability for grass surface
z = 2  # measurement height (m)
u_star = 0.3  # typical friction velocity (m/s)
T_mean = 293  # K

stability_data = []
for i, hour in enumerate(hours):
    H = energy_balance_data['grass']['H'][i]
    L = monin_obukhov_length(u_star, T_mean, H)
    zeta = stability_parameter(z, L)
    psi_m, psi_h = stability_correction(zeta)
    
    stability_data.append({
        'hour': hour,
        'H': H,
        'L': L,
        'zeta': zeta,
        'psi_m': psi_m,
        'psi_h': psi_h
    })

stability_df = pd.DataFrame(stability_data)
print("Stability analysis complete!")
print(f"\nStability regimes:")
print(f"Unstable (zeta < 0): {(stability_df['zeta'] < -0.01).sum()} hours")
print(f"Neutral (|zeta| < 0.01): {(np.abs(stability_df['zeta']) < 0.01).sum()} hours")
print(f"Stable (zeta > 0): {(stability_df['zeta'] > 0.01).sum()} hours")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Monin-Obukhov length
ax = axes[0]
L_plot = stability_df['L'].copy()
L_plot = np.clip(L_plot, -500, 500)  # Clip for visualization
ax.plot(stability_df['hour'], L_plot, 'purple', linewidth=2)
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Hour of Day', fontsize=11)
ax.set_ylabel('M-O Length (m)', fontsize=11)
ax.set_title('Monin-Obukhov Length', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Stability parameter
ax = axes[1]
ax.plot(stability_df['hour'], stability_df['zeta'], 'brown', linewidth=2)
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.axhline(y=-0.1, color='blue', linestyle=':', alpha=0.5, label='Unstable')
ax.axhline(y=0.1, color='red', linestyle=':', alpha=0.5, label='Stable')
ax.set_xlabel('Hour of Day', fontsize=11)
ax.set_ylabel('Stability Parameter ζ = z/L', fontsize=11)
ax.set_title('Atmospheric Stability', fontsize=12, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Stability corrections
ax = axes[2]
ax.plot(stability_df['hour'], stability_df['psi_m'], linewidth=2, label='Momentum (ψ_m)')
ax.plot(stability_df['hour'], stability_df['psi_h'], linewidth=2, label='Heat (ψ_h)')
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Hour of Day', fontsize=11)
ax.set_ylabel('Stability Correction', fontsize=11)
ax.set_title('Stability Correction Functions', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 3: Boundary Layer Development

The planetary boundary layer (PBL) height varies diurnally based on surface heating:

$\frac{dh}{dt} = \frac{H}{\rho c_p \Delta\theta}$

where h is the PBL height and Δθ is the temperature jump at the PBL top.

In [ ]:
def boundary_layer_growth(hours, H_flux, initial_height=100, delta_theta=3):
    """
    Simulate boundary layer height evolution.
    
    Parameters:
    - hours: time array
    - H_flux: sensible heat flux array (W/m²)
    - initial_height: initial PBL height (m)
    - delta_theta: temperature jump at PBL top (K)
    
    Returns:
    - pbl_height: boundary layer height (m)
    """
    pbl_height = np.zeros_like(hours)
    pbl_height[0] = initial_height
    
    for i in range(1, len(hours)):
        dt = (hours[i] - hours[i-1]) * 3600  # Convert to seconds
        
        if H_flux[i] > 10:  # Growing PBL
            dh_dt = H_flux[i] / (AIR_DENSITY * SPECIFIC_HEAT * delta_theta)
            pbl_height[i] = pbl_height[i-1] + dh_dt * dt
        else:  # Decaying PBL or stable conditions
            # Simplified decay
            decay_rate = 0.0001  # 1/s
            pbl_height[i] = pbl_height[i-1] * np.exp(-decay_rate * dt)
            pbl_height[i] = max(pbl_height[i], 100)  # Minimum height
    
    return pbl_height

# Calculate PBL height for grass surface
pbl_height = boundary_layer_growth(hours, energy_balance_data['grass']['H'])

print(f"PBL height evolution calculated!")
print(f"Minimum PBL height: {pbl_height.min():.0f} m")
print(f"Maximum PBL height: {pbl_height.max():.0f} m")

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Sensible heat flux
ax1.plot(hours, energy_balance_data['grass']['H'], 'r-', linewidth=2)
ax1.fill_between(hours, 0, energy_balance_data['grass']['H'], 
                  where=energy_balance_data['grass']['H']>0, 
                  alpha=0.3, color='red', label='Heating')
ax1.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax1.set_ylabel('Sensible Heat Flux (W/m²)', fontsize=12)
ax1.set_title('Surface Heating and Boundary Layer Development', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# PBL height
ax2.plot(hours, pbl_height, 'b-', linewidth=2.5)
ax2.fill_between(hours, 0, pbl_height, alpha=0.3, color='blue')
ax2.set_xlabel('Hour of Day', fontsize=12)
ax2.set_ylabel('PBL Height (m)', fontsize=12)
ax2.set_title('Planetary Boundary Layer Height', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Add annotations
max_idx = np.argmax(pbl_height)
ax2.annotate(f'Max: {pbl_height[max_idx]:.0f} m', 
             xy=(hours[max_idx], pbl_height[max_idx]),
             xytext=(hours[max_idx]+2, pbl_height[max_idx]+200),
             arrowprops=dict(arrowstyle='->', color='black'),
             fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

## Exercises

### Exercise 1: Energy Balance Partitioning
Calculate the daytime average Bowen ratio (H/LE) for each surface type.

In [ ]:
# Your code here
print("Daytime Average Bowen Ratio by Surface Type:")
print("="*50)

for surface in surfaces:
    data = energy_balance_data[surface]
    daytime = data['Rn'] > 0
    
    H_day = data['H'][daytime]
    LE_day = data['LE'][daytime]
    
    # Avoid division by zero
    bowen = np.mean(H_day / (LE_day + 1e-6))
    
    print(f"{surface.capitalize():15s}: {bowen:.2f}")

print("\nInterpretation:")
print("- High Bowen ratio (desert): sensible heat dominates")
print("- Low Bowen ratio (water): latent heat dominates")

### Exercise 2: Stability Transition Times
Identify the times when the atmosphere transitions from stable to unstable and back.

In [ ]:
# Your code here
# Find transitions
stable = stability_df['zeta'] > 0.01
unstable = stability_df['zeta'] < -0.01

# Find transition points
stable_to_unstable = []
unstable_to_stable = []

for i in range(1, len(stability_df)):
    if stable.iloc[i-1] and unstable.iloc[i]:
        stable_to_unstable.append(stability_df['hour'].iloc[i])
    elif unstable.iloc[i-1] and stable.iloc[i]:
        unstable_to_stable.append(stability_df['hour'].iloc[i])

print("Stability Transitions:")
print("="*50)
if stable_to_unstable:
    print(f"Stable → Unstable: {stable_to_unstable[0]:.1f} hours")
if unstable_to_stable:
    print(f"Unstable → Stable: {unstable_to_stable[0]:.1f} hours")
print("\nThese transitions occur around sunrise and sunset.")

## Discussion Questions

1. **Why does the PBL height peak in the afternoon?**
   - Maximum surface heating in afternoon
   - Strongest convective mixing
   - Time needed to mix heat through the boundary layer

2. **What determines atmospheric stability?**
   - Sign of sensible heat flux
   - Positive H → warming → instability
   - Negative H → cooling → stability

3. **How do different surfaces affect local climate?**
   - Energy partitioning affects temperature and humidity
   - Desert: hot and dry (high H, low LE)
   - Water/vegetation: moderate temperature, high humidity (high LE)

4. **Why is the energy balance closure often imperfect in measurements?**
   - Measurement errors and uncertainties
   - Spatial heterogeneity
   - Unaccounted energy storage terms
   - Different footprints of measurement systems

## Summary

In this lab, you have:
- Integrated all components of the surface energy balance
- Analyzed turbulent fluxes and atmospheric stability
- Simulated boundary layer development
- Explored land-atmosphere feedbacks

## Course Summary

Throughout these five labs, you have learned about:
1. **Lab 1**: Basic concepts and energy balance introduction
2. **Lab 2**: Radiation components and surface temperature
3. **Lab 3**: Soil moisture and hydrological processes
4. **Lab 4**: Vegetation and evapotranspiration
5. **Lab 5**: Integrated surface fluxes and atmospheric interactions

These concepts form the foundation for understanding land surface processes and their role in the Earth system!